# Verification of the dimming extraction procedure

In [3]:
%matplotlib notebook 

import numpy as np

import matplotlib.ticker
import matplotlib.pyplot as plt
import matplotlib.colors as colors

import sunpy.map
import sunpy.data.sample
from sunpy.net import Fido, attrs as a
from sunpy.coordinates import Helioprojective, RotatedSunFrame, transform_with_sun_center
from sunpy.io import write_file
from sunpy.time import parse_time

import astropy.units as u
from astropy.coordinates import SkyCoord
from astropy.wcs import WCS
from astropy.utils.data import get_pkg_data_filename
from astropy.io import fits

from reproject import reproject_interp, reproject_adaptive, reproject_exact
from aiapy.calibrate import normalize_exposure, register, update_pointing, fix_observer_location
from aiapy.calibrate.util import get_pointing_table
from scipy import ndimage
import os, sys
import copy
from PIL import Image
import datetime
import math
  
import cv2

%run ./functions.ipynb

## Event choice

In [4]:
# # SEPTEMBER 06 2011
# data_folder = "E:/DATA/POLIMI/THESIS/.fits_files/Event-2011-09-06/raw_img/"
# event = "E:/DATA/POLIMI/THESIS/.fits_files/Event-2011-09-06"
# # cropping coordinates:
# top=[650, 650]
# bottom=[-150, -150]


# # JUNE 14 2012
# data_folder = "F:/DATA/POLIMI/THESIS/.fits_files/Final-3_days/Event-2012-06-14/raw_img/"
# event = "F:/DATA/POLIMI/THESIS/.fits_files/Final-3_days/Event-2012-06-14"
# # cropping coordinates:
# top=[400, 200] # top right
# bottom=[-600, -800] # bottom left [x, y]

# MARCH 07 2012
data_folder = "F:/DATA/POLIMI/THESIS/.fits_files/NEW_diffrot/Event-2012-03-07/raw_img/"
event = "F:/DATA/POLIMI/THESIS/.fits_files/NEW_diffrot/Event-2012-03-07"
# cropping coordinates:
top=[ -100, 850] # top right
bottom=[-900, -250] # bottom left [x, y]


# # MARCH 08 2019
# data_folder = "E:/DATA/POLIMI/THESIS/.fits_files/Final-3_days/Event-2019-03-08/raw_img/"
# event = "E:/DATA/POLIMI/THESIS/.fits_files/Final-3_days/Event-2019-03-08"
# # cropping coordinates:
# top=[300, 600] # top right
# bottom=[-300, 0] # bottom left

## Preprocessing of the data
- Loading the desired images
- Selecting the first one as the base map
- Pre-process each image (aligning the solar North, normalizing by the exposure time, resampling)
- Obtaining the differentially rotated, base difference and log base ration images
- Save the above mentioned maps as FITS files

In [ ]:
""" The first map (to be used as basemap) is in the folder
"E:/DATA/POLIMI/THESIS/.fits_files/Event-XXXX-XX-XX/raw_img/" """ 


listdir = os.listdir(data_folder) #show elements in the folder
number= len(listdir)
print('Number of elements: ', number)

###############################################################
#                     BASE MAP

basemap_file=data_folder+listdir[0]

# base Map
base_raw=sunpy.map.Map(basemap_file)
# print(base_raw.meta)

if base_raw.meta['exptime']>2.9 and base_raw.meta['exptime']<3:
    #Calibration
    base_updated_pointing= update_pointing(base_raw)
    base_registered = register(base_updated_pointing)
    print('\n\n', base_registered.meta)

    basemap= normalize_exposure(base_registered)

    # Resampling
    new_dimensions = [2048, 2048] * u.pixel
    basemap = basemap.resample(new_dimensions)

    #Time
    base_time=sunpy.time.parse_time(basemap.meta['date-obs'])
    time_string=base_time.strftime("%Y%m%d-%H-%M-%S")
    
    basemap.save(event+'/basemap-{string}.fits'.format(string=time_string), overwrite=True)

    # print('\n\n', basemap.data)
    
else:
    raise ValueError('The basemap exposure time is not valid.\nChange basemap.')

In [6]:
from astropy.coordinates import SkyCoord
from astropy.wcs import WCS

import sunpy.map
from sunpy.coordinates import Helioprojective, propagate_with_solar_surface

def diff_rot_new(mapname, time_diff, basemap):
    
    #NOTE: to go back in time, time_diff MUST be negative
    in_time = mapname.date
    out_time = in_time + time_diff
    
    # The output frame is shifted in time for an observer at Earth (i.e., about five degrees offset in
    # heliographic longitude compared to the location of AIA in the original observation).
    out_frame = Helioprojective(observer='earth', obstime= out_time, rsun= mapname.meta['rsun_ref']*u.m)
#     rot_frame = RotatedSunFrame(base=out_frame, rotated_time=in_time)
    
    # WCS converts cdelt and cunit from arcsec to deg
    out_wcs = basemap.wcs
#     print('\nout_wcs:\n', out_wcs,'\nout_wcs type: ', type(out_wcs))
#     out_wcs.coordinate_frame = rot_frame

    # Reproject the map from the input frame to the output frame.
    with propagate_with_solar_surface():
        out_warp = mapname.reproject_to(out_wcs)
    
    
    out_warp.meta["date-obs"]=mapname.meta["date-obs"]
##     print(out_warp.meta["date-obs"])
    if mapname.name[0:3]=='AIA':
        out_warp.meta["waveunit"] = mapname.meta["waveunit"]
        out_warp.meta["exptime"] = mapname.meta['exptime']
        out_warp.meta["wavelnth"] = mapname.meta['wavelnth']
        out_warp.meta["instrume"]= mapname.meta['instrume']
        out_warp.meta["telescop"]= mapname.meta['telescop']
        out_warp.meta["rsun_obs"]= mapname.meta['rsun_obs']
        out_warp.meta["pc1_1"]= mapname.meta['pc1_1']
        out_warp.meta["pc1_2"]= mapname.meta['pc1_2']
        out_warp.meta["pc2_1"]= mapname.meta['pc2_1']
        out_warp.meta["pc2_2"]= mapname.meta['pc2_2']
        
    
    out_warp.meta["cdelt1"]=mapname.meta['cdelt1']
    out_warp.meta["cdelt2"]=mapname.meta['cdelt2']
    out_warp.meta["cunit1"]=mapname.meta['cunit1']
    out_warp.meta["cunit2"]=mapname.meta['cunit2']
    
    return out_warp

In [ ]:
""" The maps are in the folder
"F:/DATA/POLIMI/THESIS/.fits_files/Event-XXXX-XX-XX/raw_img/"
""" 

###############################################################
#                         EVENT MAPS
pointer=0

# for z in range(1,number): #avoid using the basemap image!
for z in range(340, 350):
# for z in range(360, number):
    file_name = listdir[z]
    map_file=data_folder+file_name
    map_el= sunpy.map.Map(map_file)

    if map_el.meta['exptime']>2.9 and map_el.meta['exptime']<3:

    #IMAGE PREPARATION
#         print(map_el.meta)
        # Calibration
        m_updated_pointing= update_pointing(map_el)
        
#         m_observer_fixed= fix_observer_location(m_updated_pointing)
        m_registered = register(m_updated_pointing)
#         m_registered = register(map_el)
#         print('\n\n', m_registered.meta)

        # Normalization
#         m_normalized = sunpy.map.Map(m_registered.data/m_registered.exposure_time.to(u.s).value,
#                                      m_registered.meta)
        m_normalized = normalize_exposure(m_registered)

        # Resampling
        imageprep=m_normalized.resample(new_dimensions)
#         print('\n\n', imageprep.meta)
        
        # Differential Rotation (back to reference time)
        t=sunpy.time.parse_time(imageprep.meta['date-obs'])
        time_diff= -(t-base_time).to(u.min)
        out_warp=diff_rot_new(imageprep, time_diff, basemap)

        x = out_warp.data
        x=x[~np.isnan(x)]
#         print(x)
        print('Max: ', np.max(x),'\nMin: ', np.min(x))

    # BASE DIFFERENCE   
        # Base Difference
        diff = out_warp.data - basemap.data
        meta = out_warp.meta
        diff_map = sunpy.map.Map(diff, meta)
#         print(meta)
#         print(diff)
    # LOGARITHMIC BASE DIFFERENCE            
        # Logarithmic Base Ratio image of base map and rotated image
#         diff_ratio = out_warp.data / basemap.data
#         log_diff = np.log(diff_ratio)
        meta_ratio = out_warp.meta
        log_diff = np.log(out_warp.data) - np.log(basemap.data)
        log_diff_map = sunpy.map.Map(log_diff, meta_ratio)

        time_map=parse_time(map_el.meta["date-obs"])
        time_string=time_map.strftime("%Y%m%d-%H-%M-%S")
        

    else:
        #print(f'Exposure time of the map at {map_el.meta["date-obs"]} is out of the defined range')
        pointer=pointer+1
        print(f'{map_el.meta["date-obs"]} exposure time is outside the set range')
        print(f'Exposure time: {map_el.meta["exptime"]}')
        #pass
        
print(pointer, ' maps are out of time range')

## Cropping
- Load the images (BD and LBR)
- Crop the desired area, i.e. where the dimming is present

In [3]:
data_folder = event+'/BD/'
listdir_BD = os.listdir(data_folder) #show elements in the folder
# print(listdir_BD[1:])
number_BD =len(listdir_BD)

BD_event=[]
for fits in listdir_BD: # exclude the folder
    BD_event.append(data_folder+fits)
BDcrop_seq= sunpy.map.Map(BD_event, sequence=True)

data_folder = event+'/LBR/'
listdir_LBR = os.listdir(data_folder) #show elements in the folder
# print(listdir_LBR[1:])
LBR_event=[]
for fits in listdir_LBR: # exclude the folder
    LBR_event.append(data_folder+fits)
LBRcrop_seq= sunpy.map.Map(LBR_event, sequence=True)
# LBR_seq.peek()

In [7]:
data_folder = event+'/DiffRot/'
listdir_DR = os.listdir(data_folder) #show elements in the folder
# print(listdir_DR[1:])
DR_event=[]
for fits in listdir_DR[1:]: # exclude the folder
# for fits in listdir_DR[289:]: # exclude the folder
    DR_event.append(data_folder+fits)
DR_seq= sunpy.map.Map(DR_event, sequence=True)

In [ ]:
############################################################################################
#                             CROPPING

for i in range( 100,300): #, len(DR_seq)): 
    time_map=parse_time(DR_seq[i].meta["date-obs"])
#     time_map=parse_time(BD_seq[i].meta["date-obs"])
    
    time_string=time_map.strftime("%Y%m%d-%H-%M-%S")

#     BD_crop_map=cropping(BD_seq[i], top, bottom, vmin=-7, vmax=7)
#     LBR_crop_map=cropping(LBR_seq[i], top, bottom, vmin=-1, vmax=1)
    Diffrot_map=cropping(DR_seq[i], top, bottom, vmin=-1, vmax=1)

## Histograms
- Plot histograms

In [3]:
BD_crop=sunpy.map.Map(event+'/BD/BD crop/*.fits', sequence=True)
#BD_crop.peek()
LBR_crop=sunpy.map.Map(event+'/LBR/LBR crop/*.fits', sequence=True)

In [ ]:
############################################################################################
#                             HISTOGRAMS

pre_event_BD = BD_crop[0]
# Obtain .data array
pre_data = pre_event_BD.data[~np.isnan(pre_event_BD.data)]
# num_bins=500
# bins_BDpre = np.linspace(np.min(pre_data), np.max(pre_data), num_bins)
# bins_BDpre = np.linspace(-50, 50, 80)
# print(bins_BDpre)
bins_BDpre= int((np.max(pre_data)-np.min(pre_data)))

pre_event_LBR = LBR_crop[0]
pre_LBRdata = pre_event_LBR.data[~np.isnan(pre_event_LBR.data)]
num_bins= 5000
num_bins= int((np.max(pre_LBRdata)-np.min(pre_LBRdata))*30)
print(num_bins)
# num_bins=int((np.max(pre_LBRdata)-np.min(pre_LBRdata))*100)
bins_LBR = np.linspace(np.min(pre_LBRdata), np.max(pre_LBRdata), num_bins)
# bins_LBR = np.linspace(-1, 1, num_bins)
# bins_LBR = np.linspace(-0.4, 0.4, 50)
# bins_LBR= 1000
# bins_LBR= int(np.max(pre_LBRdata)-np.min(pre_LBRdata))*10
# print(bins_LBR)

for i in range(1, len(BD_crop)):
# for i in range(31, 32):

    fig=plt.figure(figsize=(7,10))
    fig.suptitle(f'Histograms\n {BD_crop[i].meta["date-obs"][:19]}', fontsize=20, weight='bold')

    ###########################################
    ax1=fig.add_subplot(2,1,1)
    
    #Pre-event Base Difference 
    hist, bin_edges = np.histogram(pre_data, bins=bins_BDpre)

    # Note: use .ravel() here to avoid matplotlib interpreting each row
    # in the array as a different dataset to histogram
    plt.hist(pre_event_BD.data.ravel(), bins= bins_BDpre, label=f'Pre-event {pre_event_BD.meta["date-obs"][11:16]}', 
             histtype='step', color='black')
    plt.xlabel('Intensity')
    plt.axvline(np.mean(pre_data), label='pre-event mean={:.2f}'.format(np.mean(pre_data)), color='green')
    plt.legend()

    # Event Base Difference
    # Obtain .data array
    x = BD_crop[i].data[~np.isnan(BD_crop[i].data)]
#     bins = np.linspace(np.min(x), np.max(x), num_bins)
    bins_BD= int((np.max(x)-np.min(x))/2)
    hist, bin_edges = np.histogram(x, bins= bins_BD)
    plt.hist(BD_crop[i].data.ravel(), bins= bins_BD, label=f'Event: day {BD_crop[i].meta["date-obs"][8:10]} time {BD_crop[i].meta["date-obs"][11:16]}', 
             histtype='step', color='red')
    plt.title('Base Difference')
    plt.xlabel('Intensity [DN]')
    plt.ylabel('Number of pixels')
    plt.axvline(np.mean(x), label='event mean={:.2f}'.format(np.mean(x)), color='green')
    plt.legend()
    plt.xlim([-45,45])

    ############################################
    ax2=fig.add_subplot(2,1,2)
    ## Pre-event Logarithmic Base Ratio 
    hist, bin_edges = np.histogram(pre_event_LBR.data, bins= bins_LBR)
    plt.hist(pre_event_LBR.data.ravel(), bins= bins_LBR, label=f'Pre-event {pre_event_LBR.meta["date-obs"][11:16]}', 
             histtype='step', color='black')
    plt.axvline(np.mean(pre_LBRdata), label='pre-event mean={:.2f}'.format(np.mean(pre_LBRdata)), color='green')

    ## Event Logarithmic Base Ratio
    x_LBR =  LBR_crop[i].data[~np.isnan(LBR_crop[i].data)]
    num_bins_LBR= int((np.max(x_LBR)-np.min(x_LBR))*10)
    bins_LBR_event = np.linspace(np.min(x_LBR), np.max(x_LBR), num_bins_LBR)
#     bins_LBR_event = bins_LBR
#     bins = np.linspace(np.min(x_LBR), np.max(x_LBR), num_bins)
#     bins_LBR_event = int((np.max(x_LBR)-np.min(x_LBR)))
    hist2, bin_edges2 = np.histogram(LBR_crop[i].data, bins= bins_LBR_event)

    plt.hist(LBR_crop[i].data.ravel(), bins= bins_LBR_event, label=f'Event: day {LBR_crop[i].meta["date-obs"][8:10]} at {LBR_crop[i].meta["date-obs"][11:16]}', 
             histtype='step', color='red')
    plt.title('Logarithmic Base Ratio')
    plt.xlabel('Logarithmic Intensity')
    plt.ylabel('Number of pixels')
    plt.axvline(np.mean(x_LBR), label='event mean={:.2f}'.format(np.mean(x_LBR)), color='green')
    #plt.yscale('log')
    plt.xlim([-2.5,2.5])
    plt.legend()

    plt.savefig(event+'/images/Histograms/Histogram {number}.png'.format(number=i), format= 'png')

print('Pre event: ', np.max(pre_data)-np.min(pre_data),'\nEvent: ', np.max(x)-np.min(x))
print('Pre event LBR: ', np.max(pre_LBRdata)-np.min(pre_LBRdata),'\nEvent LBR: ', np.max(x_LBR)-np.min(x_LBR))
#     # Print a table of significant parameters
#     parameters= [[np.mean(pre_data), np.mean(x)], [np.min(pre_data), np.min(x)],
#                  [ np.max(pre_data), np.max(x)]]
#     param_name= ["mean", "minimum", "maximum"]
#     event_list= ['Pre-event', 'Event']

#     # Solution 2: 
#     table_BD=pd.DataFrame(parameters, index=param_name, columns= (event_list))
#     print(table_BD)

